<a href="https://colab.research.google.com/github/Sounakray2003/Asmadiya-tech/blob/main/KYC_OCR_AWS_Textract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install -q ultralytics wandb gradio boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
import os
import numpy as np
import cv2
import time
import json
import re
import boto3
import gradio as gr
import wandb
from ultralytics import YOLO
from google.colab import drive, userdata

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download Haar cascade for face detection
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

In [ ]:
# Set AWS credentials from Colab Secrets
try:
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    os.environ['AWS_DEFAULT_REGION'] = userdata.get('AWS_DEFAULT_REGION')
    print("AWS credentials loaded from Colab Secrets")
except Exception as e:
    print(f"Error loading Colab Secrets: {str(e)}")
    raise Exception("Failed to load AWS credentials from Colab Secrets. Check Secrets tab.")

AWS credentials loaded from Colab Secrets


In [ ]:
# Initialize AWS Textract client
try:
    textract = boto3.client('textract', region_name='ap-south-1')
    print("Textract client initialized successfully")
except Exception as e:
    print(f"Error initializing Textract client: {str(e)}")
    raise Exception("Failed to initialize Textract client.")

Textract client initialized successfully


In [ ]:


# Set up WandB
wandb.login()
wandb.init(
    project="document_classification_ocr",
    name="yolov11_classification_with_ocr",
    config={
        "img_size": 224
    }
)
config = wandb.config

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rajputayushsingh07042004 (rajputayushsingh07042004-asmadiya-technologies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Load pre-trained YOLOv11 model
model_path = '/content/drive/MyDrive/document_classifier_yolo12.pt'
try:
    model = YOLO(model_path)
    print(f"Loaded model from: {model_path}")
except Exception as e:
    print(f"Error loading model: {e}")
    raise Exception("Failed to load the model. Ensure the file exists and is a valid YOLO model.")


Loaded model from: /content/drive/MyDrive/document_classifier_yolo12.pt


In [ ]:
# Preprocess image for OCR
def preprocess_image_for_ocr(image):
    try:
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
        # Noise reduction
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # Binarization
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        print("Image preprocessed for OCR")
        return binary
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None

In [ ]:
# Blur detection
def is_blurry(image, threshold=100):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        wandb.log({'laplacian_variance': laplacian_var})
        return laplacian_var < threshold
    except Exception as e:
        print(f"Error in blur detection: {str(e)}")
        return True  # Assume blurry on error

In [ ]:
# Enhanced text cleaning function
def clean_extracted_text(text, field_type="general"):
    """Clean extracted text based on field type"""
    if not text:
        return ""

    text = text.strip()

    # Remove common OCR artifacts
    text = re.sub(r'[^\w\s/.-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    if field_type == "date":
        # Extract date patterns
        date_match = re.search(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{4}\b', text)
        if date_match:
            return date_match.group(0)
    elif field_type == "number":
        # Extract only digits and spaces
        return re.sub(r'[^\d\s]', '', text).strip()
    elif field_type == "name":
        # Clean name - remove extra spaces and common prefixes
        text = re.sub(r'(?i)^(mr\.?|ms\.?|mrs\.?|dr\.?)\s*', '', text)
        # Remove trailing unwanted text that might be concatenated
        text = re.sub(r'\s+(will|shall|and|the|of|for|in|at|to|from).*$', '', text, flags=re.IGNORECASE)
        # Keep only alphabetic characters and spaces
        text = re.sub(r'[^A-Za-z\s]', ' ', text)
        # Remove extra spaces and capitalize properly
        text = ' '.join(text.split())
        return text.upper() if text else ""

    return text

In [ ]:
# Extract key-value pairs from Textract response with enhanced logic
def extract_key_value_pairs(textract_response, doc_type):
    """Extract key-value pairs using layout-based logic for each document type"""
    key_value_pairs = {}

    # Get all text blocks with their bounding boxes for layout analysis
    text_blocks = []
    line_blocks = []

    try:
        for block in textract_response['Blocks']:
            if block['BlockType'] == 'WORD' and 'Text' in block:
                text_blocks.append({
                    'text': block['Text'],
                    'bbox': block['Geometry']['BoundingBox'],
                    'confidence': block.get('Confidence', 0)
                })
            elif block['BlockType'] == 'LINE' and 'Text' in block:
                line_blocks.append({
                    'text': block['Text'],
                    'bbox': block['Geometry']['BoundingBox'],
                    'confidence': block.get('Confidence', 0)
                })
    except Exception as e:
        print(f"Error processing Textract blocks: {str(e)}")
        return {}

    # Sort blocks by position (top to bottom, left to right)
    line_blocks.sort(key=lambda x: (x['bbox']['Top'], x['bbox']['Left']))

    # Full text for fallback regex matching
    full_text = '\n'.join([block['text'] for block in line_blocks])

    if doc_type == 'Aadhaar':
        key_value_pairs = extract_aadhaar_data(line_blocks, full_text)
    elif doc_type == 'PAN':
        key_value_pairs = extract_pan_data(line_blocks, full_text)
    elif doc_type == 'Passport':
        key_value_pairs = extract_passport_data(line_blocks, full_text)

    return key_value_pairs

In [ ]:
def extract_aadhaar_data(line_blocks, full_text):
    """Extract Aadhaar card data using specific layout rules"""
    data = {}

    try:
        # Find indices of key landmarks for positional extraction
        govt_india_index = -1
        dob_index = -1

        # First pass: Find Government of India and DOB positions
        for i, block in enumerate(line_blocks):
            block_text = block['text'].strip()

            # Look for "Government of India" text
            if re.search(r'(?i)government\s+of\s+india', block_text):
                govt_india_index = i
                print(f"Found 'Government of India' at index {i}: {block_text}")

            # Look for DOB/YOB patterns
            if re.search(r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|Year\s*of\s*Birth|जन्म\s*तिथि)', block_text):
                dob_index = i
                print(f"Found DOB pattern at index {i}: {block_text}")

        # Enhanced name extraction using positional logic
        name_found = False

        if govt_india_index != -1 and dob_index != -1 and govt_india_index < dob_index:
            # Look for name between Government of India and DOB
            print(f"Searching for name between indices {govt_india_index} and {dob_index}")

            for i in range(govt_india_index + 1, dob_index):
                if i >= len(line_blocks):
                    break

                block_text = line_blocks[i]['text'].strip()

                # Skip empty blocks or blocks with unwanted patterns
                if not block_text or len(block_text) < 2:
                    continue

                # Skip blocks containing numbers, addresses, or other non-name content
                if re.search(r'\d{4}\s*\d{4}\s*\d{4}|\d{4}|\bPin\b|\bpincode\b|address|DOB|YOB|Gender|Male|Female|VID|UID', block_text, re.IGNORECASE):
                    continue

                # Skip government/authority related text
                if re.search(r'(?i)government|india|unique|identification|authority|aadhaar|uid|uidai', block_text):
                    continue

                # Look for English alphabetic names (2+ words preferred, but accept single names too)
                if re.match(r'^[A-Za-z\s]{2,}$', block_text):
                    # Additional validation: should not be common non-name words
                    if not re.search(r'(?i)^(the|and|or|of|in|at|to|for|with|by)$', block_text.strip()):
                        candidate_name = clean_extracted_text(block_text, "name")
                        if candidate_name and len(candidate_name) >= 2:
                            data['name'] = candidate_name
                            name_found = True
                            print(f"Found name at index {i}: {candidate_name}")
                            break

        # Fallback method 1: If positional method fails, look for name just above any DOB line
        if not name_found and dob_index > 0:
            print("Using fallback method 1: Looking for name just above DOB")

            # Check 1-3 lines above DOB
            for offset in range(1, min(4, dob_index + 1)):
                check_index = dob_index - offset
                if check_index < 0:
                    break

                block_text = line_blocks[check_index]['text'].strip()

                # Same validation as above
                if (re.match(r'^[A-Za-z\s]{2,}$', block_text) and
                    not re.search(r'\d|government|india|unique|identification|authority|aadhaar|uid|address|pin', block_text, re.IGNORECASE) and
                    not re.search(r'(?i)^(the|and|or|of|in|at|to|for|with|by)$', block_text.strip())):

                    candidate_name = clean_extracted_text(block_text, "name")
                    if candidate_name and len(candidate_name) >= 2:
                        data['name'] = candidate_name
                        name_found = True
                        print(f"Found name using fallback method 1: {candidate_name}")
                        break

        # Fallback method 2: If still not found, use regex on full text
        if not name_found:
            print("Using fallback method 2: Regex pattern matching")

            # Look for name patterns in context
            name_patterns = [
                # Pattern: Government of India followed by name followed by DOB/address
                r'(?i)government\s+of\s+india.*?\n\s*([A-Z][A-Za-z\s]{1,49}?)\s*\n.*?(?:DOB|YOB|Date|जन्म|Address)',
                # Pattern: Name followed by DOB with reasonable distance
                r'(?i)([A-Z][A-Za-z\s]{1,49}?)\s*\n.*?(?:DOB|YOB|Date\s*of\s*Birth|जन्म\s*तिथि)',
                # Pattern: Government of India followed immediately by name
                r'(?i)government\s+of\s+india\s*?\n\s*([A-Z][A-Za-z\s]{1,49}?)',
                # General pattern for names (stricter validation)
                r'(?i)\n\s*([A-Z][A-Za-z\s]{3,49}?)\s*\n(?!.*(?:government|india|authority|unique|identification))'
            ]

            for pattern in name_patterns:
                matches = re.finditer(pattern, full_text, re.MULTILINE | re.DOTALL)
                for match in matches:
                    candidate_name = clean_extracted_text(match.group(1), "name")

                    # Additional validation
                    if (candidate_name and
                        len(candidate_name.split()) >= 1 and  # At least 1 word
                        len(candidate_name) >= 3 and  # At least 3 characters
                        not re.search(r'(?i)government|india|authority|unique|identification|address|pincode|male|female', candidate_name)):

                        data['name'] = candidate_name
                        name_found = True
                        print(f"Found name using regex fallback: {candidate_name}")
                        break

                if name_found:
                    break

        if not name_found:
            print("Warning: Could not extract name using any method")


        # DOB/YOB extraction (keeping existing logic)
        dob_patterns = [
            r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|Year\s*of\s*Birth)\s*?:?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4})',
            r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|Year\s*of\s*Birth)\s*?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4})',
            r'(?i)जन्म\s*तिथि[:/\s]*(\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4})',
            r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{4})\b'
        ]

        for pattern in dob_patterns:
            match = re.search(pattern, full_text)
            if match:
                data['dob'] = clean_extracted_text(match.group(1), "date")
                break

        # Gender extraction with enhanced logic
        gender_found = False

        # Method 1: Look for Gender label
        gender_match = re.search(r'(?i)(?:Gender|लिंग)\s*[:/]?\s*(Male|Female|Transgender|M|F)', full_text)
        if not gender_match:
            # Check for format like /Gender(Male/Female)
            gender_match = re.search(r'/Gender\s*\(\s*(Male|Female)\s*\)', full_text)

        if gender_match:
            gender = gender_match.group(1).upper()
            if gender in ['M', 'MALE']:
                data['gender'] = 'Male'
            elif gender in ['F', 'FEMALE']:
                data['gender'] = 'Female'
            else:
                data['gender'] = gender_match.group(1)
            gender_found = True
            print(f"Found gender using label method: {data['gender']}")

        # Method 2: If no Gender label found, look for gender beneath DOB
        if not gender_found and dob_index != -1:
            print("Using fallback method: Looking for gender beneath DOB")

            # Check 1-3 lines below DOB
            for offset in range(1, min(4, len(line_blocks) - dob_index)):
                check_index = dob_index + offset
                if check_index >= len(line_blocks):
                    break

                block_text = line_blocks[check_index]['text'].strip()

                # Look for gender patterns (English part only)
                gender_patterns = [
                    r'(?i)\b(Male|Female|Transgender|M|F)\b',
                    r'(?i)/(Male|Female)',  # Format like "पुरुष/Male"
                    r'(?i)(Male|Female)/',  # Format like "Male/पुरुष"
                ]

                for pattern in gender_patterns:
                    gender_match = re.search(pattern, block_text)
                    if gender_match:
                        gender = gender_match.group(1).upper()
                        if gender in ['M', 'MALE']:
                            data['gender'] = 'Male'
                        elif gender in ['F', 'FEMALE']:
                            data['gender'] = 'Female'
                        elif gender == 'TRANSGENDER':
                            data['gender'] = 'Transgender'
                        else:
                            data['gender'] = gender_match.group(1)

                        gender_found = True
                        print(f"Found gender beneath DOB at index {check_index}: {data['gender']}")
                        break

                if gender_found:
                    break

        # Method 3: Final fallback - regex search in full text for standalone gender
        if not gender_found:
            print("Using final fallback: Regex search for standalone gender")

            # Look for gender in context after DOB
            fallback_patterns = [
                r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|जन्म\s*तिथि).*?\n.*?(Male|Female|Transgender)',
                r'(?i)\b(Male|Female|Transgender)\b(?!\s*(?:Name|DOB|Address))',  # Standalone gender not followed by field names
                r'(?i)/(Male|Female)(?!/)',  # Gender in bilingual format
                r'(?i)(Male|Female)/(?!Name)',  # Gender in bilingual format
            ]

            for pattern in fallback_patterns:
                gender_match = re.search(pattern, full_text, re.MULTILINE | re.DOTALL)
                if gender_match:
                    gender = gender_match.group(1).upper()
                    if gender in ['M', 'MALE']:
                        data['gender'] = 'Male'
                    elif gender in ['F', 'FEMALE']:
                        data['gender'] = 'Female'
                    elif gender == 'TRANSGENDER':
                        data['gender'] = 'Transgender'
                    else:
                        data['gender'] = gender_match.group(1)

                    gender_found = True
                    print(f"Found gender using final fallback: {data['gender']}")
                    break

        if not gender_found:
            print("Warning: Could not extract gender using any method")

        # Aadhaar number and VID extraction (keeping existing logic)
        # Look for 12-digit Aadhaar number first
        aadhaar_match = re.search(r'\b(\d{4}\s*\d{4}\s*\d{4})\b', full_text)
        if aadhaar_match:
            aadhaar_num = aadhaar_match.group(1).replace(' ', '')
            # Check if it's masked (contains X or is partially hidden)
            if 'X' not in aadhaar_match.group(0) and len(aadhaar_num) == 12:
                data['aadhaar_number'] = aadhaar_num
            else:
                # Look for VID (16-digit number)
                vid_match = re.search(r'(?i)VID\s*:?\s*(\d{16})', full_text)
                if vid_match:
                    data['vid'] = vid_match.group(1)


    except Exception as e:
        print(f"Error extracting Aadhaar data: {str(e)}")

    return data

In [ ]:
def extract_pan_data(line_blocks, full_text):
    """Extract PAN card data using specific layout rules"""
    data = {}

    try:
        # PAN number extraction
        pan_match = re.search(r'\b([A-Z]{5}\d{4}[A-Z])\b', full_text)
        if pan_match:
            data['pan_number'] = pan_match.group(1)

        # Enhanced name extraction using line-by-line analysis
        name_found = False
        father_name_found = False

        for i, block in enumerate(line_blocks):
            block_text = block['text'].strip()

            # Look for Name label
            if re.search(r'(?i)नाम\s*/\s*Name', block_text) and not name_found:
                # Get the next line as the name
                if i + 1 < len(line_blocks):
                    name_text = line_blocks[i + 1]['text'].strip()
                    # Clean and validate the name
                    if re.match(r'^[A-Z\s]{2,50}$', name_text) and not re.search(r'\d|Father|पिता', name_text):
                        data['name'] = clean_extracted_text(name_text, "name")
                        name_found = True

            # Look for Father's Name label
            elif re.search(r'(?i)पिता\s*का\s*नाम\s*/\s*Father\'?s?\s*Name', block_text) and not father_name_found:
                # Get the next line as father's name
                if i + 1 < len(line_blocks):
                    father_text = line_blocks[i + 1]['text'].strip()
                    # Clean and validate the father's name
                    if re.match(r'^[A-Z\s]{2,50}$', father_text) and not re.search(r'\d|Date|DOB', father_text):
                        data['father_name'] = clean_extracted_text(father_text, "name")
                        father_name_found = True

        # Fallback regex patterns if line-by-line analysis fails
        if not name_found:
            name_patterns = [
                r'(?i)नाम\s*/\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|$)',
                r'(?i)/Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|पिता|Father)',
                r'(?i)Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|पिता|Father)'
            ]

            for pattern in name_patterns:
                match = re.search(pattern, full_text, re.MULTILINE | re.DOTALL)
                if match:
                    candidate_name = clean_extracted_text(match.group(1), "name")
                    # Validate it's not father's name or other text
                    if not re.search(r'Father|FATHER|पिता|DOB|Date', candidate_name):
                        data['name'] = candidate_name
                        break

        if not father_name_found:
            father_patterns = [
                r'(?i)पिता\s*का\s*नाम\s*/\s*Father\'?s?\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|$)',
                r'(?i)Father\'?s?\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|Date|DOB|जन्म)',
                r'(?i)/Father\'?s?\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|Date|DOB)'
            ]

            for pattern in father_patterns:
                match = re.search(pattern, full_text, re.MULTILINE | re.DOTALL)
                if match:
                    candidate_father = clean_extracted_text(match.group(1), "name")
                    # Validate it's not other text
                    if not re.search(r'DOB|Date|Birth|जन्म|तिथि', candidate_father):
                        data['father_name'] = candidate_father
                        break

        # DOB extraction
        dob_patterns = [
            r'(?i)(?:जन्म\s*तिथि\s*/\s*Date\s*of\s*Birth|DOB)\s*\n\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'(?i)/Date\s*of\s*Birth\s*\n\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{4})\b'
        ]

        for pattern in dob_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['dob'] = clean_extracted_text(match.group(1), "date")
                break

    except Exception as e:
        print(f"Error extracting PAN data: {str(e)}")

    return data

In [ ]:
def extract_passport_data(line_blocks, full_text):
    """Extract Passport data using specific layout rules"""
    data = {}

    try:
        # Passport number extraction
        passport_patterns = [
            r'(?i)/Passport\s*No\.?\s*\n\s*([A-Z]\d{7})',
            r'(?i)Passport\s*No\.?\s*:?\s*([A-Z]\d{7})',
            r'\b([A-Z]\d{7})\b'
        ]

        for pattern in passport_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['passport_number'] = match.group(1)
                break

        # Surname extraction
        surname_patterns = [
            r'(?i)/Surname\s*\n\s*([A-Z\s]+)',
            r'(?i)Surname\s*:?\s*([A-Z][A-Za-z\s]+)'
        ]

        for pattern in surname_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['surname'] = clean_extracted_text(match.group(1), "name")
                break

        # Given name extraction
        given_name_patterns = [
            r'(?i)/?Given\s*Names?\s*\(\s*s\s*\)?\s*\n\s*([A-Z\s]+)',  # Handles "/Given Name(s)" or "/Given Names"
            r'(?i)/?Given\s*Names?\s*\n\s*([A-Z\s]+)',  # Handles "/Given Name" or "/Given Names"
            r'(?i)Given\s*Names?\s*\(\s*s\s*\)?\s*:?\s*([A-Z][A-Za-z\s]+)',  # Handles "Given Name(s):" or "Given Names:"
            r'(?i)Given\s*Names?\s*:?\s*([A-Z][A-Za-z\s]+)'  # Handles "Given Name:" or "Given Names:"
        ]

        for pattern in given_name_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['given_names'] = clean_extracted_text(match.group(1), "name")
                break

        # Nationality extraction - extract only English part
        nationality_patterns = [
            r'(?i)/Nationality\s*\n\s*(?:.*/)?(INDIAN|AMERICAN|BRITISH|[A-Z]+)',
            r'(?i)Nationality\s*:?\s*(?:.*/)?(INDIAN|AMERICAN|BRITISH|[A-Z]+)'
        ]

        for pattern in nationality_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['nationality'] = match.group(1)
                break

        # Sex extraction
        sex_patterns = [
            r'(?i)/Sex\s*\n\s*([MF])',
            r'(?i)Sex\s*:?\s*([MF]|Male|Female)'
        ]

        for pattern in sex_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                sex = match.group(1).upper()
                data['sex'] = 'M' if sex in ['M', 'MALE'] else 'F' if sex in ['F', 'FEMALE'] else sex
                break

        # DOB extraction
        dob_patterns = [
            r'(?i)/Date\s*of\s*Birth\s*\n\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'(?i)Date\s*of\s*Birth\s*:?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{4})\b'
        ]

        for pattern in dob_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['dob'] = clean_extracted_text(match.group(1), "date")
                break

    except Exception as e:
        print(f"Error extracting Passport data: {str(e)}")

    return data

In [ ]:

# Extract user photo from document
def extract_user_photo(image, doc_type):
    """Extract user photo based on document type and layout"""
    try:
        if doc_type == 'Aadhaar':
            # Left-center area for Aadhaar
            h, w = image.shape[:2]
            photo_region = image[int(h*0.2):int(h*0.8), int(w*0.05):int(w*0.35)]
        elif doc_type == 'PAN':
            # Top-left area for PAN
            h, w = image.shape[:2]
            photo_region = image[int(h*0.1):int(h*0.6), int(w*0.05):int(w*0.3)]
        elif doc_type == 'Passport':
            # Left side for Passport
            h, w = image.shape[:2]
            photo_region = image[int(h*0.2):int(h*0.7), int(w*0.05):int(w*0.25)]
        else:
            return None

        # Verify it contains a human face using basic face detection
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(photo_region, cv2.COLOR_RGB2GRAY) if len(photo_region.shape) == 3 else photo_region

        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        if len(faces) > 0:
            return photo_region
        else:
            print(f"No clear human face detected in {doc_type} photo region")
            return None

    except Exception as e:
        print(f"Error extracting photo: {str(e)}")
        return None

In [ ]:
# Classify and extract document data
def classify_and_extract_document(selected_type, image):
    # Check for blurriness
    if is_blurry(image):
        return "Please upload a clearer image.", {}

    # Classify with YOLOv11
    try:
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) if len(image.shape) == 3 else image
        predictions = model.predict(image_bgr, imgsz=config.img_size, verbose=False)[0]
        predicted_class_idx = predictions.probs.top1
        predicted_label = ['Aadhaar', 'PAN', 'Passport'][predicted_class_idx]
        confidence = predictions.probs.top1conf.item()

        wandb.log({
            'selected_type': selected_type,
            'predicted_label': predicted_label,
            'confidence': confidence
        })
    except Exception as e:
        print(f"Error during classification: {str(e)}")
        return "Error during classification.", {}

    if predicted_label != selected_type or confidence < 0.3:
        return f"Invalid document. Detected as {predicted_label} with {confidence:.3f} confidence.", {}

    # Extract user photo
    user_photo = extract_user_photo(image, predicted_label)

    # Preprocess for OCR
    processed_image = preprocess_image_for_ocr(image)
    if processed_image is None:
        return "Error preprocessing image for OCR.", {}

    # Encode image for Textract
    try:
        _, buffer = cv2.imencode('.png', processed_image)
        image_bytes = buffer.tobytes()
        print(f"Image encoded successfully, size: {len(image_bytes)} bytes")
    except Exception as e:
        print(f"Error encoding image: {str(e)}")
        return "Error encoding image for Textract.", {}

    # Call AWS Textract
    try:
        response = textract.analyze_document(
            Document={'Bytes': image_bytes},
            FeatureTypes=['FORMS']
        )
        print("Textract response received")
    except Exception as e:
        print(f"Textract error: {str(e)}")
        return f"Textract error: {str(e)}", {}

    # Extract key-value pairs using enhanced logic
    key_value_pairs = extract_key_value_pairs(response, predicted_label)

    # Save image and extracted data
    output_dir = f'/content/drive/MyDrive/Output/{predicted_label}'
    os.makedirs(output_dir, exist_ok=True)
    timestamp = int(time.time())
    image_path = os.path.join(output_dir, f"{predicted_label}_{timestamp}.png")
    try:
        cv2.imwrite(image_path, image_bgr)
        print(f"Image saved to: {image_path}")
    except Exception as e:
        print(f"Error saving image: {str(e)}")

    # Save user photo if extracted
    if user_photo is not None:
        photo_path = os.path.join(output_dir, f"{predicted_label}_{timestamp}_photo.png")
        try:
            cv2.imwrite(photo_path, user_photo)
            key_value_pairs['user_photo_path'] = photo_path
            print(f"User photo saved to: {photo_path}")
        except Exception as e:
            print(f"Error saving user photo: {str(e)}")

    json_path = os.path.join(output_dir, f"{predicted_label}_{timestamp}.json")
    try:
        with open(json_path, 'w') as f:
            json.dump(key_value_pairs, f, indent=4)
        print(f"JSON saved to: {json_path}")
    except Exception as e:
        print(f"Error saving JSON: {str(e)}")

    wandb.log({'extracted_fields': key_value_pairs})

    # Format output message
    result_message = f"Document accepted: {predicted_label} (Confidence: {confidence:.3f})\n\nExtracted Data:\n"
    for key, value in key_value_pairs.items():
        if value and key != 'user_photo_path':  # Don't show empty fields or photo path in main output
            result_message += f"{key.replace('_', ' ').title()}: {value}\n"

    return result_message, key_value_pairs

In [ ]:
# Gradio interface
iface = gr.Interface(
    fn=classify_and_extract_document,
    inputs=[
        gr.Dropdown(choices=['Aadhaar', 'PAN', 'Passport'], label="Select Document Type"),
        gr.Image(type="numpy")
    ],
    outputs=["text", "json"],
    title="Enhanced Document Classifier and OCR - YOLOv11 with AWS Textract",
    description="Upload a clear image of your Aadhaar, PAN, or Passport for accurate data extraction."
)

In [ ]:
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b9000a4cc4680bcb92.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


No clear human face detected in PAN photo region
Image preprocessed for OCR
Image encoded successfully, size: 29968 bytes
Textract response received
Image saved to: /content/drive/MyDrive/Output/PAN/PAN_1755581430.png
JSON saved to: /content/drive/MyDrive/Output/PAN/PAN_1755581430.json
No clear human face detected in PAN photo region
Image preprocessed for OCR
Image encoded successfully, size: 29968 bytes
Textract response received
Image saved to: /content/drive/MyDrive/Output/PAN/PAN_1755581571.png
JSON saved to: /content/drive/MyDrive/Output/PAN/PAN_1755581571.json
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b9000a4cc4680bcb92.gradio.live


In [ ]:
 # Finish WandB run
wandb.finish()

confidence,███▁█
laplacian_variance,▂▂▁█▂
confidence,1.0
laplacian_variance,1030.88221
predicted_label,Passport
selected_type,Passport


In [ ]:
# Install required packages
!pip install -q ultralytics wandb gradio boto3

import os
import numpy as np
import cv2
import time
import json
import re
import boto3
import gradio as gr
import wandb
from ultralytics import YOLO
from google.colab import drive, userdata

# Mount Google Drive
drive.mount('/content/drive')

# Download Haar cascade for face detection
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

# Set AWS credentials from Colab Secrets
try:
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    os.environ['AWS_DEFAULT_REGION'] = userdata.get('AWS_DEFAULT_REGION')
    print("AWS credentials loaded from Colab Secrets")
except Exception as e:
    print(f"Error loading Colab Secrets: {str(e)}")
    raise Exception("Failed to load AWS credentials from Colab Secrets. Check Secrets tab.")

# Initialize AWS Textract client
try:
    textract = boto3.client('textract', region_name='ap-south-1')
    print("Textract client initialized successfully")
except Exception as e:
    print(f"Error initializing Textract client: {str(e)}")
    raise Exception("Failed to initialize Textract client.")

# Set up WandB
wandb.login()
wandb.init(
    project="document_classification_ocr",
    name="yolov11_classification_with_ocr",
    config={
        "img_size": 224
    }
)
config = wandb.config

# Load pre-trained YOLOv11 model
model_path = '/content/drive/MyDrive/document_classifier_yolo12.pt'
try:
    model = YOLO(model_path)
    print(f"Loaded model from: {model_path}")
except Exception as e:
    print(f"Error loading model: {e}")
    raise Exception("Failed to load the model. Ensure the file exists and is a valid YOLO model.")

# Preprocess image for OCR
def preprocess_image_for_ocr(image):
    try:
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
        # Noise reduction
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # Binarization
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        print("Image preprocessed for OCR")
        return binary
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None

# Blur detection
def is_blurry(image, threshold=100):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        wandb.log({'laplacian_variance': laplacian_var})
        return laplacian_var < threshold
    except Exception as e:
        print(f"Error in blur detection: {str(e)}")
        return True  # Assume blurry on error

# Enhanced text cleaning function
def clean_extracted_text(text, field_type="general"):
    """Clean extracted text based on field type"""
    if not text:
        return ""

    text = text.strip()

    # Remove common OCR artifacts
    text = re.sub(r'[^\w\s/.-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    if field_type == "date":
        # Extract date patterns
        date_match = re.search(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{4}\b', text)
        if date_match:
            return date_match.group(0)
    elif field_type == "number":
        # Extract only digits and spaces
        return re.sub(r'[^\d\s]', '', text).strip()
    elif field_type == "name":
        # Clean name - remove extra spaces and common prefixes
        text = re.sub(r'(?i)^(mr\.?|ms\.?|mrs\.?|dr\.?)\s*', '', text)
        # Remove trailing unwanted text that might be concatenated
        text = re.sub(r'\s+(will|shall|and|the|of|for|in|at|to|from).*$', '', text, flags=re.IGNORECASE)
        # Keep only alphabetic characters and spaces
        text = re.sub(r'[^A-Za-z\s]', ' ', text)
        # Remove extra spaces and capitalize properly
        text = ' '.join(text.split())
        return text.upper() if text else ""

    return text

# Extract key-value pairs from Textract response with enhanced logic
def extract_key_value_pairs(textract_response, doc_type):
    """Extract key-value pairs using layout-based logic for each document type"""
    key_value_pairs = {}

    # Get all text blocks with their bounding boxes for layout analysis
    text_blocks = []
    line_blocks = []

    try:
        for block in textract_response['Blocks']:
            if block['BlockType'] == 'WORD' and 'Text' in block:
                text_blocks.append({
                    'text': block['Text'],
                    'bbox': block['Geometry']['BoundingBox'],
                    'confidence': block.get('Confidence', 0)
                })
            elif block['BlockType'] == 'LINE' and 'Text' in block:
                line_blocks.append({
                    'text': block['Text'],
                    'bbox': block['Geometry']['BoundingBox'],
                    'confidence': block.get('Confidence', 0)
                })
    except Exception as e:
        print(f"Error processing Textract blocks: {str(e)}")
        return {}

    # Sort blocks by position (top to bottom, left to right)
    line_blocks.sort(key=lambda x: (x['bbox']['Top'], x['bbox']['Left']))

    # Full text for fallback regex matching
    full_text = '\n'.join([block['text'] for block in line_blocks])

    if doc_type == 'Aadhaar':
        key_value_pairs = extract_aadhaar_data(line_blocks, full_text)
    elif doc_type == 'PAN':
        key_value_pairs = extract_pan_data(line_blocks, full_text)
    elif doc_type == 'Passport':
        key_value_pairs = extract_passport_data(line_blocks, full_text)

    return key_value_pairs

def extract_aadhaar_data(line_blocks, full_text):
    """Extract Aadhaar card data using specific layout rules"""
    data = {}

    try:
        # Find indices of key landmarks for positional extraction
        govt_india_index = -1
        dob_index = -1

        # First pass: Find Government of India and DOB positions
        for i, block in enumerate(line_blocks):
            block_text = block['text'].strip()

            # Look for "Government of India" text
            if re.search(r'(?i)government\s+of\s+india', block_text):
                govt_india_index = i
                print(f"Found 'Government of India' at index {i}: {block_text}")

            # Look for DOB/YOB patterns
            if re.search(r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|Year\s*of\s*Birth|जन्म\s*तिथि)', block_text):
                dob_index = i
                print(f"Found DOB pattern at index {i}: {block_text}")

        # Enhanced name extraction using positional logic
        name_found = False

        if govt_india_index != -1 and dob_index != -1 and govt_india_index < dob_index:
            # Look for name between Government of India and DOB
            print(f"Searching for name between indices {govt_india_index} and {dob_index}")

            for i in range(govt_india_index + 1, dob_index):
                if i >= len(line_blocks):
                    break

                block_text = line_blocks[i]['text'].strip()

                # Skip empty blocks or blocks with unwanted patterns
                if not block_text or len(block_text) < 2:
                    continue

                # Skip blocks containing numbers, addresses, or other non-name content
                if re.search(r'\d{4}\s*\d{4}\s*\d{4}|\d{4}|\bPin\b|\bpincode\b|address|DOB|YOB|Gender|Male|Female|VID|UID', block_text, re.IGNORECASE):
                    continue

                # Skip government/authority related text
                if re.search(r'(?i)government|india|unique|identification|authority|aadhaar|uid|uidai', block_text):
                    continue

                # Look for English alphabetic names (2+ words preferred, but accept single names too)
                if re.match(r'^[A-Za-z\s]{2,}$', block_text):
                    # Additional validation: should not be common non-name words
                    if not re.search(r'(?i)^(the|and|or|of|in|at|to|for|with|by)$', block_text.strip()):
                        candidate_name = clean_extracted_text(block_text, "name")
                        if candidate_name and len(candidate_name) >= 2:
                            data['name'] = candidate_name
                            name_found = True
                            print(f"Found name at index {i}: {candidate_name}")
                            break

        # Fallback method 1: If positional method fails, look for name just above any DOB line
        if not name_found and dob_index > 0:
            print("Using fallback method 1: Looking for name just above DOB")

            # Check 1-3 lines above DOB
            for offset in range(1, min(4, dob_index + 1)):
                check_index = dob_index - offset
                if check_index < 0:
                    break

                block_text = line_blocks[check_index]['text'].strip()

                # Same validation as above
                if (re.match(r'^[A-Za-z\s]{2,}$', block_text) and
                    not re.search(r'\d|government|india|unique|identification|authority|aadhaar|uid|address|pin', block_text, re.IGNORECASE) and
                    not re.search(r'(?i)^(the|and|or|of|in|at|to|for|with|by)$', block_text.strip())):

                    candidate_name = clean_extracted_text(block_text, "name")
                    if candidate_name and len(candidate_name) >= 2:
                        data['name'] = candidate_name
                        name_found = True
                        print(f"Found name using fallback method 1: {candidate_name}")
                        break

        # Fallback method 2: If still not found, use regex on full text
        if not name_found:
            print("Using fallback method 2: Regex pattern matching")

            # Look for name patterns in context
            name_patterns = [
                # Pattern: Government of India followed by name followed by DOB/address
                r'(?i)government\s+of\s+india.*?\n\s*([A-Z][A-Za-z\s]{1,49}?)\s*\n.*?(?:DOB|YOB|Date|जन्म|Address)',
                # Pattern: Name followed by DOB with reasonable distance
                r'(?i)([A-Z][A-Za-z\s]{1,49}?)\s*\n.*?(?:DOB|YOB|Date\s*of\s*Birth|जन्म\s*तिथि)',
                # Pattern: Government of India followed immediately by name
                r'(?i)government\s+of\s+india\s*?\n\s*([A-Z][A-Za-z\s]{1,49}?)',
                # General pattern for names (stricter validation)
                r'(?i)\n\s*([A-Z][A-Za-z\s]{3,49}?)\s*\n(?!.*(?:government|india|authority|unique|identification))'
            ]

            for pattern in name_patterns:
                matches = re.finditer(pattern, full_text, re.MULTILINE | re.DOTALL)
                for match in matches:
                    candidate_name = clean_extracted_text(match.group(1), "name")

                    # Additional validation
                    if (candidate_name and
                        len(candidate_name.split()) >= 1 and  # At least 1 word
                        len(candidate_name) >= 3 and  # At least 3 characters
                        not re.search(r'(?i)government|india|authority|unique|identification|address|pincode|male|female', candidate_name)):

                        data['name'] = candidate_name
                        name_found = True
                        print(f"Found name using regex fallback: {candidate_name}")
                        break

                if name_found:
                    break

        if not name_found:
            print("Warning: Could not extract name using any method")


        # DOB/YOB extraction (keeping existing logic)
        dob_patterns = [
            r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|Year\s*of\s*Birth)\s*?:?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4})',
            r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|Year\s*of\s*Birth)\s*?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4})',
            r'(?i)जन्म\s*तिथि[:/\s]*(\d{1,2}[/-]\d{1,2}[/-]\d{4}|\d{4})',
            r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{4})\b'
        ]

        for pattern in dob_patterns:
            match = re.search(pattern, full_text)
            if match:
                data['dob'] = clean_extracted_text(match.group(1), "date")
                break

        # Gender extraction with enhanced logic
        gender_found = False

        # Method 1: Look for Gender label
        gender_match = re.search(r'(?i)(?:Gender|लिंग)\s*[:/]?\s*(Male|Female|Transgender|M|F)', full_text)
        if not gender_match:
            # Check for format like /Gender(Male/Female)
            gender_match = re.search(r'/Gender\s*\(\s*(Male|Female)\s*\)', full_text)

        if gender_match:
            gender = gender_match.group(1).upper()
            if gender in ['M', 'MALE']:
                data['gender'] = 'Male'
            elif gender in ['F', 'FEMALE']:
                data['gender'] = 'Female'
            else:
                data['gender'] = gender_match.group(1)
            gender_found = True
            print(f"Found gender using label method: {data['gender']}")

        # Method 2: If no Gender label found, look for gender beneath DOB
        if not gender_found and dob_index != -1:
            print("Using fallback method: Looking for gender beneath DOB")

            # Check 1-3 lines below DOB
            for offset in range(1, min(4, len(line_blocks) - dob_index)):
                check_index = dob_index + offset
                if check_index >= len(line_blocks):
                    break

                block_text = line_blocks[check_index]['text'].strip()

                # Look for gender patterns (English part only)
                gender_patterns = [
                    r'(?i)\b(Male|Female|Transgender|M|F)\b',
                    r'(?i)/(Male|Female)',  # Format like "पुरुष/Male"
                    r'(?i)(Male|Female)/',  # Format like "Male/पुरुष"
                ]

                for pattern in gender_patterns:
                    gender_match = re.search(pattern, block_text)
                    if gender_match:
                        gender = gender_match.group(1).upper()
                        if gender in ['M', 'MALE']:
                            data['gender'] = 'Male'
                        elif gender in ['F', 'FEMALE']:
                            data['gender'] = 'Female'
                        elif gender == 'TRANSGENDER':
                            data['gender'] = 'Transgender'
                        else:
                            data['gender'] = gender_match.group(1)

                        gender_found = True
                        print(f"Found gender beneath DOB at index {check_index}: {data['gender']}")
                        break

                if gender_found:
                    break

        # Method 3: Final fallback - regex search in full text for standalone gender
        if not gender_found:
            print("Using final fallback: Regex search for standalone gender")

            # Look for gender in context after DOB
            fallback_patterns = [
                r'(?i)(?:DOB|YOB|Date\s*of\s*Birth|जन्म\s*तिथि).*?\n.*?(Male|Female|Transgender)',
                r'(?i)\b(Male|Female|Transgender)\b(?!\s*(?:Name|DOB|Address))',  # Standalone gender not followed by field names
                r'(?i)/(Male|Female)(?!/)',  # Gender in bilingual format
                r'(?i)(Male|Female)/(?!Name)',  # Gender in bilingual format
            ]

            for pattern in fallback_patterns:
                gender_match = re.search(pattern, full_text, re.MULTILINE | re.DOTALL)
                if gender_match:
                    gender = gender_match.group(1).upper()
                    if gender in ['M', 'MALE']:
                        data['gender'] = 'Male'
                    elif gender in ['F', 'FEMALE']:
                        data['gender'] = 'Female'
                    elif gender == 'TRANSGENDER':
                        data['gender'] = 'Transgender'
                    else:
                        data['gender'] = gender_match.group(1)

                    gender_found = True
                    print(f"Found gender using final fallback: {data['gender']}")
                    break

        if not gender_found:
            print("Warning: Could not extract gender using any method")

        # Aadhaar number and VID extraction (keeping existing logic)
        # Look for 12-digit Aadhaar number first
        aadhaar_match = re.search(r'\b(\d{4}\s*\d{4}\s*\d{4})\b', full_text)
        if aadhaar_match:
            aadhaar_num = aadhaar_match.group(1).replace(' ', '')
            # Check if it's masked (contains X or is partially hidden)
            if 'X' not in aadhaar_match.group(0) and len(aadhaar_num) == 12:
                data['aadhaar_number'] = aadhaar_num
            else:
                # Look for VID (16-digit number)
                vid_match = re.search(r'(?i)VID\s*:?\s*(\d{16})', full_text)
                if vid_match:
                    data['vid'] = vid_match.group(1)


    except Exception as e:
        print(f"Error extracting Aadhaar data: {str(e)}")

    return data

def extract_pan_data(line_blocks, full_text):
    """Extract PAN card data using specific layout rules"""
    data = {}

    try:
        # PAN number extraction
        pan_match = re.search(r'\b([A-Z]{5}\d{4}[A-Z])\b', full_text)
        if pan_match:
            data['pan_number'] = pan_match.group(1)

        # Enhanced name extraction using line-by-line analysis
        name_found = False
        father_name_found = False

        for i, block in enumerate(line_blocks):
            block_text = block['text'].strip()

            # Look for Name label
            if re.search(r'(?i)नाम\s*/\s*Name', block_text) and not name_found:
                # Get the next line as the name
                if i + 1 < len(line_blocks):
                    name_text = line_blocks[i + 1]['text'].strip()
                    # Clean and validate the name
                    if re.match(r'^[A-Z\s]{2,50}$', name_text) and not re.search(r'\d|Father|पिता', name_text):
                        data['name'] = clean_extracted_text(name_text, "name")
                        name_found = True

            # Look for Father's Name label
            elif re.search(r'(?i)पिता\s*का\s*नाम\s*/\s*Father\'?s?\s*Name', block_text) and not father_name_found:
                # Get the next line as father's name
                if i + 1 < len(line_blocks):
                    father_text = line_blocks[i + 1]['text'].strip()
                    # Clean and validate the father's name
                    if re.match(r'^[A-Z\s]{2,50}$', father_text) and not re.search(r'\d|Date|DOB', father_text):
                        data['father_name'] = clean_extracted_text(father_text, "name")
                        father_name_found = True

        # Fallback regex patterns if line-by-line analysis fails
        if not name_found:
            name_patterns = [
                r'(?i)नाम\s*/\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|$)',
                r'(?i)/Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|पिता|Father)',
                r'(?i)Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|पिता|Father)'
            ]

            for pattern in name_patterns:
                match = re.search(pattern, full_text, re.MULTILINE | re.DOTALL)
                if match:
                    candidate_name = clean_extracted_text(match.group(1), "name")
                    # Validate it's not father's name or other text
                    if not re.search(r'Father|FATHER|पिता|DOB|Date', candidate_name):
                        data['name'] = candidate_name
                        break

        if not father_name_found:
            father_patterns = [
                r'(?i)पिता\s*का\s*नाम\s*/\s*Father\'?s?\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|$)',
                r'(?i)Father\'?s?\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|Date|DOB|जन्म)',
                r'(?i)/Father\'?s?\s*Name\s*\n\s*([A-Z][A-Z\s]{1,49}?)(?=\n|Date|DOB)'
            ]

            for pattern in father_patterns:
                match = re.search(pattern, full_text, re.MULTILINE | re.DOTALL)
                if match:
                    candidate_father = clean_extracted_text(match.group(1), "name")
                    # Validate it's not other text
                    if not re.search(r'DOB|Date|Birth|जन्म|तिथि', candidate_father):
                        data['father_name'] = candidate_father
                        break

        # DOB extraction
        dob_patterns = [
            r'(?i)(?:जन्म\s*तिथि\s*/\s*Date\s*of\s*Birth|DOB)\s*\n\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'(?i)/Date\s*of\s*Birth\s*\n\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{4})\b'
        ]

        for pattern in dob_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['dob'] = clean_extracted_text(match.group(1), "date")
                break

    except Exception as e:
        print(f"Error extracting PAN data: {str(e)}")

    return data

def extract_passport_data(line_blocks, full_text):
    """Extract Passport data using specific layout rules"""
    data = {}

    try:
        # Passport number extraction
        passport_patterns = [
            r'(?i)/Passport\s*No\.?\s*\n\s*([A-Z]\d{7})',
            r'(?i)Passport\s*No\.?\s*:?\s*([A-Z]\d{7})',
            r'\b([A-Z]\d{7})\b'
        ]

        for pattern in passport_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['passport_number'] = match.group(1)
                break

        # Surname extraction
        surname_patterns = [
            r'(?i)/Surname\s*\n\s*([A-Z\s]+)',
            r'(?i)Surname\s*:?\s*([A-Z][A-Za-z\s]+)'
        ]

        for pattern in surname_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['surname'] = clean_extracted_text(match.group(1), "name")
                break

        # Given name extraction
        given_name_patterns = [
            r'(?i)/?Given\s*Names?\s*\(\s*s\s*\)?\s*\n\s*([A-Z\s]+)',  # Handles "/Given Name(s)" or "/Given Names"
            r'(?i)/?Given\s*Names?\s*\n\s*([A-Z\s]+)',  # Handles "/Given Name" or "/Given Names"
            r'(?i)Given\s*Names?\s*\(\s*s\s*\)?\s*:?\s*([A-Z][A-Za-z\s]+)',  # Handles "Given Name(s):" or "Given Names:"
            r'(?i)Given\s*Names?\s*:?\s*([A-Z][A-Za-z\s]+)'  # Handles "Given Name:" or "Given Names:"
        ]

        for pattern in given_name_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['given_names'] = clean_extracted_text(match.group(1), "name")
                break

        # Nationality extraction - extract only English part
        nationality_patterns = [
            r'(?i)/Nationality\s*\n\s*(?:.*/)?(INDIAN|AMERICAN|BRITISH|[A-Z]+)',
            r'(?i)Nationality\s*:?\s*(?:.*/)?(INDIAN|AMERICAN|BRITISH|[A-Z]+)'
        ]

        for pattern in nationality_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['nationality'] = match.group(1)
                break

        # Sex extraction
        sex_patterns = [
            r'(?i)/Sex\s*\n\s*([MF])',
            r'(?i)Sex\s*:?\s*([MF]|Male|Female)'
        ]

        for pattern in sex_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                sex = match.group(1).upper()
                data['sex'] = 'M' if sex in ['M', 'MALE'] else 'F' if sex in ['F', 'FEMALE'] else sex
                break

        # DOB extraction
        dob_patterns = [
            r'(?i)/Date\s*of\s*Birth\s*\n\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'(?i)Date\s*of\s*Birth\s*:?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{4})',
            r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{4})\b'
        ]

        for pattern in dob_patterns:
            match = re.search(pattern, full_text, re.MULTILINE)
            if match:
                data['dob'] = clean_extracted_text(match.group(1), "date")
                break

    except Exception as e:
        print(f"Error extracting Passport data: {str(e)}")

    return data

# Extract user photo from document
def extract_user_photo(image, doc_type):
    """Extract user photo based on document type and layout"""
    try:
        if doc_type == 'Aadhaar':
            # Left-center area for Aadhaar
            h, w = image.shape[:2]
            photo_region = image[int(h*0.2):int(h*0.8), int(w*0.05):int(w*0.35)]
        elif doc_type == 'PAN':
            # Top-left area for PAN
            h, w = image.shape[:2]
            photo_region = image[int(h*0.1):int(h*0.6), int(w*0.05):int(w*0.3)]
        elif doc_type == 'Passport':
            # Left side for Passport
            h, w = image.shape[:2]
            photo_region = image[int(h*0.2):int(h*0.7), int(w*0.05):int(w*0.25)]
        else:
            return None

        # Verify it contains a human face using basic face detection
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(photo_region, cv2.COLOR_RGB2GRAY) if len(photo_region.shape) == 3 else photo_region

        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        if len(faces) > 0:
            return photo_region
        else:
            print(f"No clear human face detected in {doc_type} photo region")
            return None

    except Exception as e:
        print(f"Error extracting photo: {str(e)}")
        return None

# Classify and extract document data
def classify_and_extract_document(selected_type, image):
    # Check for blurriness
    if is_blurry(image):
        return "Please upload a clearer image.", {}

    # Classify with YOLOv11
    try:
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) if len(image.shape) == 3 else image
        predictions = model.predict(image_bgr, imgsz=config.img_size, verbose=False)[0]
        predicted_class_idx = predictions.probs.top1
        predicted_label = ['Aadhaar', 'PAN', 'Passport'][predicted_class_idx]
        confidence = predictions.probs.top1conf.item()

        wandb.log({
            'selected_type': selected_type,
            'predicted_label': predicted_label,
            'confidence': confidence
        })
    except Exception as e:
        print(f"Error during classification: {str(e)}")
        return "Error during classification.", {}

    if predicted_label != selected_type or confidence < 0.3:
        return f"Invalid document. Detected as {predicted_label} with {confidence:.3f} confidence.", {}

    # Extract user photo
    user_photo = extract_user_photo(image, predicted_label)

    # Preprocess for OCR
    processed_image = preprocess_image_for_ocr(image)
    if processed_image is None:
        return "Error preprocessing image for OCR.", {}

    # Encode image for Textract
    try:
        _, buffer = cv2.imencode('.png', processed_image)
        image_bytes = buffer.tobytes()
        print(f"Image encoded successfully, size: {len(image_bytes)} bytes")
    except Exception as e:
        print(f"Error encoding image: {str(e)}")
        return "Error encoding image for Textract.", {}

    # Call AWS Textract
    try:
        response = textract.analyze_document(
            Document={'Bytes': image_bytes},
            FeatureTypes=['FORMS']
        )
        print("Textract response received")
    except Exception as e:
        print(f"Textract error: {str(e)}")
        return f"Textract error: {str(e)}", {}

    # Extract key-value pairs using enhanced logic
    key_value_pairs = extract_key_value_pairs(response, predicted_label)

    # Save image and extracted data
    output_dir = f'/content/drive/MyDrive/Output/{predicted_label}'
    os.makedirs(output_dir, exist_ok=True)
    timestamp = int(time.time())
    image_path = os.path.join(output_dir, f"{predicted_label}_{timestamp}.png")
    try:
        cv2.imwrite(image_path, image_bgr)
        print(f"Image saved to: {image_path}")
    except Exception as e:
        print(f"Error saving image: {str(e)}")

    # Save user photo if extracted
    if user_photo is not None:
        photo_path = os.path.join(output_dir, f"{predicted_label}_{timestamp}_photo.png")
        try:
            cv2.imwrite(photo_path, user_photo)
            key_value_pairs['user_photo_path'] = photo_path
            print(f"User photo saved to: {photo_path}")
        except Exception as e:
            print(f"Error saving user photo: {str(e)}")

    json_path = os.path.join(output_dir, f"{predicted_label}_{timestamp}.json")
    try:
        with open(json_path, 'w') as f:
            json.dump(key_value_pairs, f, indent=4)
        print(f"JSON saved to: {json_path}")
    except Exception as e:
        print(f"Error saving JSON: {str(e)}")

    wandb.log({'extracted_fields': key_value_pairs})

    # Format output message
    result_message = f"Document accepted: {predicted_label} (Confidence: {confidence:.3f})\n\nExtracted Data:\n"
    for key, value in key_value_pairs.items():
        if value and key != 'user_photo_path':  # Don't show empty fields or photo path in main output
            result_message += f"{key.replace('_', ' ').title()}: {value}\n"

    return result_message, key_value_pairs

# Gradio interface
iface = gr.Interface(
    fn=classify_and_extract_document,
    inputs=[
        gr.Dropdown(choices=['Aadhaar', 'PAN', 'Passport'], label="Select Document Type"),
        gr.Image(type="numpy")
    ],
    outputs=["text", "json"],
    title="Enhanced Document Classifier and OCR - YOLOv11 with AWS Textract",
    description="Upload a clear image of your Aadhaar, PAN, or Passport for accurate data extraction."
)

iface.launch(debug=True)

 # Finish WandB run
wandb.finish()